# Multi-Agent Coding System
In this notebook, we will explore the concept of a multi-agent coding system. This system is designed to work with multiple agents, each capable of performing specific tasks. The agents will collaborate to solve complex problems and achieve their goals.

The implementation uses OpenAI's API to create three specialized agents:

1. Code Writer Agent : Transforms natural language descriptions into functional Python code, allowing users to quickly generate code snippets based on their requirements.
2. Debugger Agent : Analyzes generated code to identify potential issues, bugs, and areas for improvement, providing detailed feedback similar to a code review.
3. Documenter Agent : Creates comprehensive documentation for code snippets, making it easier to understand and maintain the generated code.

### 1. Setting Up Dependencies

In [22]:
#  imports and setup
import os
import json
import uuid
import functools
import asyncio
import nest_asyncio
from datetime import datetime
from typing import Dict, Any, Callable, List
from openai import OpenAI

# apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

### 2. MCP Framework

This cell defines the Model Context Protocol (MCP) class, which provides a standardized way for AI agents to communicate:

- The @mcp.tool decorator transforms regular methods into MCP-compliant agent tools
- It uses Python's introspection capabilities ( inspect.signature() ) to analyze function parameters
- The decorator creates a context dictionary containing agent name, task description, expected format, and parameters
- String formatting is used to populate task description templates with actual parameter values
- functools.wraps preserves the original function's metadata
- The context is passed to the wrapped function, enabling consistent tracking of agent activities

In [15]:
# Cell 2: Model Context Protocol (MCP) Implementation
class MCP:
    """Model Context Protocol module for standardized agent communication."""
    
    @staticmethod
    def tool(
        task_description: str,
        required_format: str = "json"
    ) -> Callable:
        """
        Decorator for functions that use the Model Context Protocol.
        
        Args:
            task_description (str): Template string describing the task
            required_format (str): Expected response format
            
        Returns:
            Callable: Decorated function
        """
        def decorator(func: Callable) -> Callable:
            @functools.wraps(func)
            def wrapper(self, *args, **kwargs):
                # Get function arguments
                import inspect
                sig = inspect.signature(func)
                bound_args = sig.bind(self, *args, **kwargs)
                bound_args.apply_defaults()
                
                # Create context from arguments
                arg_dict = dict(bound_args.arguments)
                arg_dict.pop('self', None)  # Remove 'self' from context
                
                # Generate task description with arguments
                formatted_task = task_description.format(**arg_dict)
                
                # Create a context dictionary
                context = {
                    "agent": self.name if hasattr(self, 'name') else type(self).__name__,
                    "task": formatted_task,
                    "format": required_format,
                    "parameters": arg_dict
                }
                
                # Call the original function with the context
                return func(self, *args, **kwargs, mcp_context=context)
            return wrapper
        return decorator

# Create an instance of the MCP module
mcp = MCP()

### 3. MCP Server Infrastructure
Implements the base MCPServer class that provides the infrastructure for tool registration and execution:

- The server maintains a dictionary of tools, each with metadata like name, description, and parameters
- register_tool() allows adding new tools to the server with their specifications
- call_tool() provides an asynchronous way to execute tools by name with parameter validation
- list_tools() returns all available tools for discovery
- The class includes placeholder methods for connection and cleanup, which could be extended for actual server implementations
- This design follows a service-oriented architecture where tools are registered services that can be discovered and called

#### Agent Workflow
1. Code Writer Agent
   
   - Takes a natural language prompt
   - Generates Python code using OpenAI
   - Stores the code as an artifact in the context

2. Debugger Agent
   
   - Analyzes code with awareness of the original task
   - Produces a debugging report
   - Maintains context from the code generation step
   
3. Documenter Agent
   
   - Creates documentation with access to both code and debug report
   - Leverages the full context history for comprehensive documentation
   - Stores documentation as a new artifact

In [16]:
# MCP Server Base Class
class MCPServer:
    """Base class for MCP servers that follows the Model Context Protocol specification."""
    
    def __init__(self):
        """Initialize the MCP server."""
        self._tools = {}
        self._server_name = "mcp_server"
    
    @property
    def name(self):
        """Return the name of the server."""
        return self._server_name
    
    async def connect(self):
        """Connect to the server."""
        # No actual connection needed for this example
        return True
    
    async def cleanup(self):
        """Clean up resources."""
        # No cleanup needed for this example
        pass
    
    def register_tool(self, tool_name, tool_desc, tool_func, parameters, required=None):
        """Register a tool with the server."""
        if required is None:
            required = []
            
        self._tools[tool_name] = {
            "name": tool_name,
            "description": tool_desc,
            "function": tool_func,
            "parameters": parameters,
            "required": required
        }
    
    async def call_tool(self, tool_name, params):
        """Call a tool by name with parameters."""
        if tool_name not in self._tools:
            raise ValueError(f"Unknown tool: {tool_name}")
        
        tool = self._tools[tool_name]
        
        # Check required parameters
        for param in tool["required"]:
            if param not in params:
                raise ValueError(f"Missing required parameter: {param} for tool {tool_name}")
        
        # Call the function
        func = tool["function"]
        return func(params)
    
    async def list_tools(self):
        """List all registered tools."""
        return list(self._tools.values())

### 4. Specialized Coding Server Setup

This cell begins the implementation of the CodingMCPServer, which specializes in coding-related tools:

- It initializes with an OpenAI API key and creates an OpenAI client instance using the new SDK format
- The server maintains a rich context structure with messages, artifacts, and metadata
- A unique session ID is generated for tracking interactions
- The _register_coding_tools() method sets up three specialized coding tools:
  1. code_writer : Generates Python code from natural language descriptions
  2. code_debugger : Analyzes code for issues and improvements
  3. code_documenter : Creates documentation for code
- Each tool registration includes detailed parameter specifications and requirements
- This design follows a plugin architecture where specialized tools can be easily added to the server

In [ ]:
# Coding MCP Server Implementation - Part 1 (Setup and Registration)
class CodingMCPServer(MCPServer):
    """MCP Server implementation specifically for coding tools."""
    
    def __init__(self, openai_api_key):
        """Initialize the coding MCP server."""
        super().__init__()
        self._server_name = "coding_server"
        self.openai_api_key = openai_api_key
        self.model = "gpt-3.5-turbo"
        self.client = OpenAI(api_key=openai_api_key)  # Create OpenAI client
        
        # Initialize context
        self.context = {
            "messages": [],
            "artifacts": {},
            "metadata": {
                "version": "1.0",
                "session_id": str(uuid.uuid4())
            }
        }
        
        # Register coding tools
        self._register_coding_tools()
    
    def _register_coding_tools(self):
        """Register all coding tools with the server."""
        # Code Writer Tool
        self.register_tool(
            tool_name="code_writer",
            tool_desc="Generates Python code from natural language descriptions",
            tool_func=self._generate_code,
            parameters={
                "prompt": {
                    "type": "string",
                    "description": "Natural language description of the code to generate"
                }
            },
            required=["prompt"]
        )
        
        # Debug Tool
        self.register_tool(
            tool_name="code_debugger",
            tool_desc="Analyzes Python code to identify issues and improvements",
            tool_func=self._debug_code,
            parameters={
                "code": {
                    "type": "string",
                    "description": "Python code to debug"
                },
                "task_description": {
                    "type": "string",
                    "description": "Original task description"
                }
            },
            required=["code"]
        )
        
        # Documenter Tool
        self.register_tool(
            tool_name="code_documenter",
            tool_desc="Creates documentation for Python code",
            tool_func=self._document_code,
            parameters={
                "code": {
                    "type": "string",
                    "description": "Python code to document"
                },
                "task_description": {
                    "type": "string",
                    "description": "Original task description"
                },
                "debug_report": {
                    "type": "string",
                    "description": "Debug report for the code"
                }
            },
            required=["code"]
        )

### 5. Context Management Methods

This cell extends the CodingMCPServer with methods for managing context and preparing messages:

- _update_context() : Records interactions between the user and assistant in the message history
  - Each message includes role, content, timestamp, and optional artifact references
  - This creates a chronological record of the conversation
- _store_artifact() : Saves generated artifacts (code, debug reports, documentation) with metadata
  - Each artifact has a unique ID, type, content, and creation timestamp
  - This allows for referencing and retrieving artifacts throughout the session
- _prepare_messages() : Formats messages for OpenAI API calls
  - Includes a system prompt that defines the assistant's role
  - Incorporates recent conversation history (limited to 5 messages for efficiency)
  - Adds the current user prompt
  - This context-aware approach enables more coherent and relevant responses

In [18]:
# Coding MCP Server Implementation - Part 2 (Context Management)
class CodingMCPServer(CodingMCPServer):  # Continuing the class definition
    def _update_context(self, role, content, artifact_id=None):
        """
        Update the MCP context with new interactions
        
        Args:
            role (str): The role of the message sender (user/assistant)
            content (str): The content of the message
            artifact_id (str, optional): ID of any generated artifact
        """
        message = {
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        }
        
        if artifact_id:
            message["artifact_id"] = artifact_id
            
        self.context["messages"].append(message)
        return message

    def _store_artifact(self, artifact_id, artifact_type, content):
        """
        Store artifacts in the MCP context
        
        Args:
            artifact_id (str): Unique identifier for the artifact
            artifact_type (str): Type of artifact (code/debug/doc)
            content (str): The artifact content
        """
        self.context["artifacts"][artifact_id] = {
            "type": artifact_type,
            "content": content,
            "created_at": datetime.now().isoformat()
        }
        return artifact_id

    def _prepare_messages(self, system_prompt, user_prompt):
        """
        Prepare messages for OpenAI API calls
        
        Args:
            system_prompt (str): The system prompt
            user_prompt (str): The user prompt
            
        Returns:
            list: Messages formatted for OpenAI API
        """
        # Create messages list with system prompt
        messages = [{"role": "system", "content": system_prompt}]
        
        # Add context messages (limited to last 5 for efficiency)
        for msg in self.context["messages"][-5:]:
            messages.append({"role": msg["role"], "content": msg["content"]})
            
        # Add the current user prompt
        messages.append({"role": "user", "content": user_prompt})
        
        return messages

### 6. AI-Powered Tool Functions

This cell implements the core AI-powered tool functions for the CodingMCPServer:

- _generate_code() : Transforms natural language descriptions into Python code
  - Uses a specialized system prompt to guide the AI's role as a code writer
  - Formats the user's prompt to focus on code generation without explanations
  - Stores the generated code as an artifact and updates the conversation context
- _debug_code() : Analyzes code to identify issues and improvements
  - Takes code and optional task description as input
  - Uses a system prompt that positions the AI as a senior software engineer
  - Provides context about the original task to ensure relevant debugging
  - Stores the debug report as an artifact for future reference
- _document_code() : Creates comprehensive documentation for code
  - Takes code, optional task description, and optional debug report
  - Uses a system prompt that positions the AI as a documentation expert
  - Incorporates the original task and debug findings for context-aware documentation
  - Stores the documentation as an artifact
- All functions use the new OpenAI API format with the client instance

In [19]:
# Cell 6: Coding MCP Server Implementation - Part 3 (Tool Functions)
class CodingMCPServer(CodingMCPServer):  # Continuing the class definition
    def _generate_code(self, params):
        """Generate code using OpenAI API."""
        prompt = params["prompt"]
        
        # Update context with user request
        self._update_context("user", prompt)
        
        # Prepare MCP-formatted messages
        system_prompt = "You are a professional Python code writer."
        user_content = f"Write a Python function for: {prompt}. Provide only the code, no explanations."
        
        messages = self._prepare_messages(system_prompt, user_content)
        
        # Send request with OpenAI API using new format
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            user=self.context["metadata"]["session_id"]
        )
        
        code = response.choices[0].message.content
        
        # Store the code as an artifact and update context
        artifact_id = f"code_{len(self.context['artifacts']) + 1}"
        self._store_artifact(artifact_id, "code", code)
        self._update_context("assistant", code, artifact_id)
        
        return code
    
    def _debug_code(self, params):
        """Debug code using OpenAI API."""
        code = params["code"]
        task_description = params.get("task_description")
        
        # Add debug request to context
        debug_request = f"Debug the following code"
        if task_description:
            debug_request += f" for task: {task_description}"
        self._update_context("user", debug_request)
        
        # Prepare MCP-formatted messages
        system_prompt = "You are a senior software engineer and code debugger."
        user_content = f"Review this Python code and identify potential issues:\n{code}"
        if task_description:
            user_content = f"This code was written for: {task_description}. {user_content}"
        
        messages = self._prepare_messages(system_prompt, user_content)
        
        # Send request with OpenAI API using new format
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            user=self.context["metadata"]["session_id"]
        )
        
        debug_report = response.choices[0].message.content
        
        # Store the debug report as an artifact and update context
        artifact_id = f"debug_{len(self.context['artifacts']) + 1}"
        self._store_artifact(artifact_id, "debug_report", debug_report)
        self._update_context("assistant", debug_report, artifact_id)
        
        return debug_report
    
    def _document_code(self, params):
        """Generate documentation using OpenAI API."""
        code = params["code"]
        task_description = params.get("task_description")
        debug_report = params.get("debug_report")
        
        # Add documentation request to context
        doc_request = "Create documentation for the code"
        self._update_context("user", doc_request)
        
        # Prepare MCP-formatted messages
        system_prompt = "You are a technical documentation expert."
        user_content = f"Create comprehensive documentation for this Python code:\n{code}"
        
        # Add relevant context to the message content
        if task_description:
            user_content = f"This code was written for: {task_description}. {user_content}"
        
        # Reference debug report if available
        if debug_report:
            user_content = f"A debug report is available: {debug_report}\nPlease consider its findings when documenting. {user_content}"
        
        messages = self._prepare_messages(system_prompt, user_content)
        
        # Send request with OpenAI API using new format
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            user=self.context["metadata"]["session_id"]
        )
        
        documentation = response.choices[0].message.content
        
        # Store the documentation as an artifact and update context
        artifact_id = f"doc_{len(self.context['artifacts']) + 1}"
        self._store_artifact(artifact_id, "documentation", documentation)
        self._update_context("assistant", documentation, artifact_id)
        
        return documentation

### 7. Agent Interface Layer

This cell implements the CodingAgents class, which serves as the user-facing interface to the coding tools:

- The class initializes with an OpenAI API key and sets up the MCP server
- It uses an event loop for handling asynchronous operations with the server
- The __del__ method ensures proper cleanup of resources when the object is destroyed
- Three MCP-decorated agent methods provide the main functionality:
  1. code_writer_agent : Transforms natural language into Python code
     - Decorated with task description "Generate Python code for: {prompt}"
     - Expected to return code format
  2. debugger_agent : Analyzes code for issues and improvements
     - Decorated with task description "Debug Python code for task: {task_description}"
     - Expected to return a report format
  3. documenter_agent : Creates documentation for code
     - Decorated with task description "Document Python code for task: {task_description}"
     - Expected to return markdown format
- Each agent method delegates to the corresponding tool in the MCP server
- The MCP decorators automatically track context and format task descriptions

In [20]:
#CodingAgents Class Implementation
class CodingAgents:
    def __init__(self, api_key):
        """
        Initialize the coding agents with OpenAI API and MCP server
        
        Args:
            api_key (str): OpenAI API key
        """
        # Setup basic properties
        self.openai_api_key = api_key  # Store API key directly
        self.model = "gpt-3.5-turbo"
        self.name = "CodingAssistant"
        
        # Initialize MCP server and connect
        self.server = CodingMCPServer(api_key)
        
        # Set up event loop for async operations
        self.loop = asyncio.get_event_loop()
        
        # Connect to the MCP server
        self.loop.run_until_complete(self.server.connect())
        
        # Use the server's context for continuity
        self.context = self.server.context
    
    def __del__(self):
        """Clean up resources when the object is destroyed."""
        # Make sure we clean up the server when done
        try:
            self.loop.run_until_complete(self.server.cleanup())
        except:
            pass
    
    @mcp.tool(
        task_description="Generate Python code for: {prompt}",
        required_format="code"
    )
    def code_writer_agent(self, prompt, mcp_context=None):
        """
        Generate code using OpenAI API with MCP
        
        Args:
            prompt (str): Coding task description
            mcp_context (Dict, optional): MCP context from decorator
        
        Returns:
            str: Generated code
        """
        # Use the MCP server to call the tool
        result = self.loop.run_until_complete(
            self.server.call_tool("code_writer", {"prompt": prompt})
        )
        return result

    @mcp.tool(
        task_description="Debug Python code for task: {task_description}",
        required_format="report"
    )
    def debugger_agent(self, code, task_description=None, mcp_context=None):
        """
        Debug code using OpenAI API with MCP
        
        Args:
            code (str): Code to debug
            task_description (str, optional): Original task description for context
            mcp_context (Dict, optional): MCP context from decorator
        
        Returns:
            str: Debugging report
        """
        # Prepare parameters
        params = {
            "code": code
        }
        if task_description:
            params["task_description"] = task_description
        
        # Use the MCP server to call the tool
        result = self.loop.run_until_complete(
            self.server.call_tool("code_debugger", params)
        )
        return result
        
    @mcp.tool(
        task_description="Document Python code for task: {task_description}",
        required_format="markdown"
    )
    def documenter_agent(self, code, task_description=None, debug_report=None, mcp_context=None):
        """
        Generate documentation using OpenAI API with MCP
        
        Args:
            code (str): Code to document
            task_description (str, optional): Original task description
            debug_report (str, optional): Debug report for context
            mcp_context (Dict, optional): MCP context from decorator
        
        Returns:
            str: Documentation
        """
        # Prepare parameters
        params = {
            "code": code
        }
        if task_description:
            params["task_description"] = task_description
        if debug_report:
            params["debug_report"] = debug_report
        
        # Use the MCP server to call the tool
        result = self.loop.run_until_complete(
            self.server.call_tool("code_documenter", params)
        )
        return result

In [21]:
# demo Function
def run_mcp_coding_demo():
    # Get API key from environment variable
    api_key = os.getenv('OPENAI_API_KEY')
    
    if not api_key:
        print("Please set OPENAI_API_KEY environment variable")
        return
    
    # Initialize the agents with MCP
    agents = CodingAgents(api_key)
    
    # Example task
    task = "Create a function that calculates the factorial of a number"
    
    print(f"🚀 Running MCP for task: {task}")
    
    # Process the task with Model Context Protocol
    code = agents.code_writer_agent(task)
    print("\n📝 Generated Code:")
    print(f"```python\n{code}\n```")
    
    debug_report = agents.debugger_agent(code, task)
    print("\n🐞 Debug Report:")
    print(debug_report)
    
    documentation = agents.documenter_agent(code, task, debug_report)
    print("\n📄 Documentation:")
    print(documentation)
    
    # List all available tools via the MCP server
    tools = agents.loop.run_until_complete(agents.server.list_tools())
    
    print("\n🧠 Context Summary:")
    print(f"Session ID: {agents.context['metadata']['session_id']}")
    print(f"Messages: {len(agents.context['messages'])}")
    print(f"Artifacts: {len(agents.context['artifacts'])}")
    print(f"Available MCP Tools: {len(tools)}")
    for tool in tools:
        print(f"  - {tool['name']}: {tool['description']}")

# Run the demo if executed directly
if __name__ == "__main__":
    run_mcp_coding_demo()

🚀 Running MCP for task: Create a function that calculates the factorial of a number

📝 Generated Code:
```python
```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
```
```

🐞 Debug Report:
The code you provided looks correct at first glance for calculating the factorial of a number. However, there are potential issues that you should consider:

1. Input validation: The current implementation assumes that the input 'n' is a non-negative integer. If a negative integer or a non-integer value is passed as input, it might result in unexpected behavior or an infinite loop.
2. Recursion depth limit: Using recursion for calculating factorials can lead to hitting the maximum recursion depth in Python for very large input values. Consider using an iterative approach for handling large values.

To address these potential issues, you may consider adding input validation checks and implementing an iterative version of the factorial function to ha

In [7]:
# Multi-Agent Coding System with Proper MCP Integration
import os
import openai
import json
import uuid
import functools
import asyncio
import nest_asyncio
from datetime import datetime
from typing import Dict, Any, Callable, List
from openai import OpenAI

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

class MCP:
    """Model Context Protocol module for standardized agent communication."""
    
    @staticmethod
    def tool(
        task_description: str,
        required_format: str = "json"
    ) -> Callable:
        """
        Decorator for functions that use the Model Context Protocol.
        
        Args:
            task_description (str): Template string describing the task
            required_format (str): Expected response format
            
        Returns:
            Callable: Decorated function
        """
        def decorator(func: Callable) -> Callable:
            @functools.wraps(func)
            def wrapper(self, *args, **kwargs):
                # Get function arguments
                import inspect
                sig = inspect.signature(func)
                bound_args = sig.bind(self, *args, **kwargs)
                bound_args.apply_defaults()
                
                # Create context from arguments
                arg_dict = dict(bound_args.arguments)
                arg_dict.pop('self', None)  # Remove 'self' from context
                
                # Generate task description with arguments
                formatted_task = task_description.format(**arg_dict)
                
                # Create a context dictionary
                context = {
                    "agent": self.name if hasattr(self, 'name') else type(self).__name__,
                    "task": formatted_task,
                    "format": required_format,
                    "parameters": arg_dict
                }
                
                # Call the original function with the context
                return func(self, *args, **kwargs, mcp_context=context)
            return wrapper
        return decorator

# Create an instance of the MCP module
mcp = MCP()

# Define MCP Server base class
class MCPServer:
    """Base class for MCP servers that follows the Model Context Protocol specification."""
    
    def __init__(self):
        """Initialize the MCP server."""
        self._tools = {}
        self._server_name = "mcp_server"
    
    @property
    def name(self):
        """Return the name of the server."""
        return self._server_name
    
    async def connect(self):
        """Connect to the server."""
        # No actual connection needed for this example
        return True
    
    async def cleanup(self):
        """Clean up resources."""
        # No cleanup needed for this example
        pass
    
    def register_tool(self, tool_name, tool_desc, tool_func, parameters, required=None):
        """Register a tool with the server."""
        if required is None:
            required = []
            
        self._tools[tool_name] = {
            "name": tool_name,
            "description": tool_desc,
            "function": tool_func,
            "parameters": parameters,
            "required": required
        }
    
    async def call_tool(self, tool_name, params):
        """Call a tool by name with parameters."""
        if tool_name not in self._tools:
            raise ValueError(f"Unknown tool: {tool_name}")
        
        tool = self._tools[tool_name]
        
        # Check required parameters
        for param in tool["required"]:
            if param not in params:
                raise ValueError(f"Missing required parameter: {param} for tool {tool_name}")
        
        # Call the function
        func = tool["function"]
        return func(params)
    
    async def list_tools(self):
        """List all registered tools."""
        return list(self._tools.values())

# Coding MCP Server implementation
class CodingMCPServer(MCPServer):
    """MCP Server implementation specifically for coding tools."""
    
    def __init__(self, openai_api_key):
        """Initialize the coding MCP server."""
        super().__init__()
        self._server_name = "coding_server"
        self.openai_api_key = openai_api_key
        self.model = "gpt-3.5-turbo"
        self.client = OpenAI(api_key=openai_api_key)  # Create OpenAI client
        
        # Initialize context
        self.context = {
            "messages": [],
            "artifacts": {},
            "metadata": {
                "version": "1.0",
                "session_id": str(uuid.uuid4())
            }
        }
        
        # Register coding tools
        self._register_coding_tools()
    
    def _register_coding_tools(self):
        """Register all coding tools with the server."""
        # Code Writer Tool
        self.register_tool(
            tool_name="code_writer",
            tool_desc="Generates Python code from natural language descriptions",
            tool_func=self._generate_code,
            parameters={
                "prompt": {
                    "type": "string",
                    "description": "Natural language description of the code to generate"
                }
            },
            required=["prompt"]
        )
        
        # Debug Tool
        self.register_tool(
            tool_name="code_debugger",
            tool_desc="Analyzes Python code to identify issues and improvements",
            tool_func=self._debug_code,
            parameters={
                "code": {
                    "type": "string",
                    "description": "Python code to debug"
                },
                "task_description": {
                    "type": "string",
                    "description": "Original task description"
                }
            },
            required=["code"]
        )
        
        # Documenter Tool
        self.register_tool(
            tool_name="code_documenter",
            tool_desc="Creates documentation for Python code",
            tool_func=self._document_code,
            parameters={
                "code": {
                    "type": "string",
                    "description": "Python code to document"
                },
                "task_description": {
                    "type": "string",
                    "description": "Original task description"
                },
                "debug_report": {
                    "type": "string",
                    "description": "Debug report for the code"
                }
            },
            required=["code"]
        )
    
    def _update_context(self, role, content, artifact_id=None):
        """
        Update the MCP context with new interactions
        
        Args:
            role (str): The role of the message sender (user/assistant)
            content (str): The content of the message
            artifact_id (str, optional): ID of any generated artifact
        """
        message = {
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        }
        
        if artifact_id:
            message["artifact_id"] = artifact_id
            
        self.context["messages"].append(message)
        return message

    def _store_artifact(self, artifact_id, artifact_type, content):
        """
        Store artifacts in the MCP context
        
        Args:
            artifact_id (str): Unique identifier for the artifact
            artifact_type (str): Type of artifact (code/debug/doc)
            content (str): The artifact content
        """
        self.context["artifacts"][artifact_id] = {
            "type": artifact_type,
            "content": content,
            "created_at": datetime.now().isoformat()
        }
        return artifact_id

    def _prepare_messages(self, system_prompt, user_prompt):
        """
        Prepare messages for OpenAI API calls
        
        Args:
            system_prompt (str): The system prompt
            user_prompt (str): The user prompt
            
        Returns:
            list: Messages formatted for OpenAI API
        """
        # Create messages list with system prompt
        messages = [{"role": "system", "content": system_prompt}]
        
        # Add context messages (limited to last 5 for efficiency)
        for msg in self.context["messages"][-5:]:
            messages.append({"role": msg["role"], "content": msg["content"]})
            
        # Add the current user prompt
        messages.append({"role": "user", "content": user_prompt})
        
        return messages
    
    def _generate_code(self, params):
        """Generate code using OpenAI API."""
        prompt = params["prompt"]
        
        # Update context with user request
        self._update_context("user", prompt)
        
        # Prepare MCP-formatted messages
        system_prompt = "You are a professional Python code writer."
        user_content = f"Write a Python function for: {prompt}. Provide only the code, no explanations."
        
        messages = self._prepare_messages(system_prompt, user_content)
        
        # Send request with OpenAI API using new format
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            user=self.context["metadata"]["session_id"]
        )
        
        code = response.choices[0].message.content
        
        # Store the code as an artifact and update context
        artifact_id = f"code_{len(self.context['artifacts']) + 1}"
        self._store_artifact(artifact_id, "code", code)
        self._update_context("assistant", code, artifact_id)
        
        return code
    
    def _debug_code(self, params):
        """Debug code using OpenAI API."""
        code = params["code"]
        task_description = params.get("task_description")
        
        # Add debug request to context
        debug_request = f"Debug the following code"
        if task_description:
            debug_request += f" for task: {task_description}"
        self._update_context("user", debug_request)
        
        # Prepare MCP-formatted messages
        system_prompt = "You are a senior software engineer and code debugger."
        user_content = f"Review this Python code and identify potential issues:\n{code}"
        if task_description:
            user_content = f"This code was written for: {task_description}. {user_content}"
        
        messages = self._prepare_messages(system_prompt, user_content)
        
        # Send request with OpenAI API using new format
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            user=self.context["metadata"]["session_id"]
        )
        
        debug_report = response.choices[0].message.content
        
        # Store the debug report as an artifact and update context
        artifact_id = f"debug_{len(self.context['artifacts']) + 1}"
        self._store_artifact(artifact_id, "debug_report", debug_report)
        self._update_context("assistant", debug_report, artifact_id)
        
        return debug_report
    
    def _document_code(self, params):
        """Generate documentation using OpenAI API."""
        code = params["code"]
        task_description = params.get("task_description")
        debug_report = params.get("debug_report")
        
        # Add documentation request to context
        doc_request = "Create documentation for the code"
        self._update_context("user", doc_request)
        
        # Prepare MCP-formatted messages
        system_prompt = "You are a technical documentation expert."
        user_content = f"Create comprehensive documentation for this Python code:\n{code}"
        
        # Add relevant context to the message content
        if task_description:
            user_content = f"This code was written for: {task_description}. {user_content}"
        
        # Reference debug report if available
        if debug_report:
            user_content = f"A debug report is available: {debug_report}\nPlease consider its findings when documenting. {user_content}"
        
        messages = self._prepare_messages(system_prompt, user_content)
        
        # Send request with OpenAI API using new format
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            user=self.context["metadata"]["session_id"]
        )
        
        documentation = response.choices[0].message.content
        
        # Store the documentation as an artifact and update context
        artifact_id = f"doc_{len(self.context['artifacts']) + 1}"
        self._store_artifact(artifact_id, "documentation", documentation)
        self._update_context("assistant", documentation, artifact_id)
        
        return documentation

class CodingAgents:
    def __init__(self, api_key):
        """
        Initialize the coding agents with OpenAI API and MCP server
        
        Args:
            api_key (str): OpenAI API key
        """
        # Setup basic properties
class CodingAgents:
    def __init__(self, api_key):
        """
        Initialize the coding agents with OpenAI API and MCP server
        
        Args:
            api_key (str): OpenAI API key
        """
        # Setup basic properties
        self.openai_api_key = api_key  # Store API key directly
        self.model = "gpt-3.5-turbo"
        self.name = "CodingAssistant"
        
        # Initialize MCP server and connect
        self.server = CodingMCPServer(api_key)
        
        # Set up event loop for async operations
        self.loop = asyncio.get_event_loop()
        
        # Connect to the MCP server
        self.loop.run_until_complete(self.server.connect())
        
        # Use the server's context for continuity
        self.context = self.server.context
    
    def __del__(self):
        """Clean up resources when the object is destroyed."""
        # Make sure we clean up the server when done
        try:
            self.loop.run_until_complete(self.server.cleanup())
        except:
            pass
    
    @mcp.tool(
        task_description="Generate Python code for: {prompt}",
        required_format="code"
    )
    def code_writer_agent(self, prompt, mcp_context=None):
        """
        Generate code using OpenAI API with MCP
        
        Args:
            prompt (str): Coding task description
            mcp_context (Dict, optional): MCP context from decorator
        
        Returns:
            str: Generated code
        """
        # Use the MCP server to call the tool
        result = self.loop.run_until_complete(
            self.server.call_tool("code_writer", {"prompt": prompt})
        )
        return result

    @mcp.tool(
        task_description="Debug Python code for task: {task_description}",
        required_format="report"
    )
    def debugger_agent(self, code, task_description=None, mcp_context=None):
        """
        Debug code using OpenAI API with MCP
        
        Args:
            code (str): Code to debug
            task_description (str, optional): Original task description for context
            mcp_context (Dict, optional): MCP context from decorator
        
        Returns:
            str: Debugging report
        """
        # Prepare parameters
        params = {
            "code": code
        }
        if task_description:
            params["task_description"] = task_description
        
        # Use the MCP server to call the tool
        result = self.loop.run_until_complete(
            self.server.call_tool("code_debugger", params)
        )
        return result
        
    @mcp.tool(
        task_description="Document Python code for task: {task_description}",
        required_format="markdown"
    )
    def documenter_agent(self, code, task_description=None, debug_report=None, mcp_context=None):
        """
        Generate documentation using OpenAI API with MCP
        
        Args:
            code (str): Code to document
            task_description (str, optional): Original task description
            debug_report (str, optional): Debug report for context
            mcp_context (Dict, optional): MCP context from decorator
        
        Returns:
            str: Documentation
        """
        # Prepare parameters
        params = {
            "code": code
        }
        if task_description:
            params["task_description"] = task_description
        if debug_report:
            params["debug_report"] = debug_report
        
        # Use the MCP server to call the tool
        result = self.loop.run_until_complete(
            self.server.call_tool("code_documenter", params)
        )
        return result

# Function to demonstrate the MCP-enhanced coding agents
def run_mcp_coding_demo():
    # Get API key from environment variable
    api_key = os.getenv('OPENAI_API_KEY')
    
    if not api_key:
        print("Please set OPENAI_API_KEY environment variable")
        return
    
    # Initialize the agents with MCP
    agents = CodingAgents(api_key)
    
    # Example task
    task = "Create a function that calculates the factorial of a number"
    
    print(f"🚀 Running MCP for task: {task}")
    
    # Process the task with Model Context Protocol
    code = agents.code_writer_agent(task)
    print("\n📝 Generated Code:")
    print(f"```python\n{code}\n```")
    
    debug_report = agents.debugger_agent(code, task)
    print("\n🐞 Debug Report:")
    print(debug_report)
    
    documentation = agents.documenter_agent(code, task, debug_report)
    print("\n📄 Documentation:")
    print(documentation)
    
    # List all available tools via the MCP server
    tools = agents.loop.run_until_complete(agents.server.list_tools())
    
    print("\n🧠 Context Summary:")
    print(f"Session ID: {agents.context['metadata']['session_id']}")
    print(f"Messages: {len(agents.context['messages'])}")
    print(f"Artifacts: {len(agents.context['artifacts'])}")
    print(f"Available MCP Tools: {len(tools)}")
    for tool in tools:
        print(f"  - {tool['name']}: {tool['description']}")

# Run the demo if executed directly
if __name__ == "__main__":
    run_mcp_coding_demo()

🚀 Running MCP for task: Create a function that calculates the factorial of a number

📝 Generated Code:
```python
```python
def factorial(num):
    if num == 0:
        return 1
    else:
        return num * factorial(num - 1)
```
```

🐞 Debug Report:
The provided code for calculating the factorial of a number looks correct and should work as expected. The base case is properly handled when num equals 0, and the recursive call correctly calculates the factorial for num - 1 until the base case is reached.

However, it's important to note that this implementation may encounter a "RecursionError" for very large values of num due to exceeding the maximum recursion depth in Python. You may want to consider using a non-recursive approach or use tail recursion optimization if handling large values is a concern.

📄 Documentation:
### Function: factorial

This Python function calculates the factorial of a given number.

#### Parameters:
- `num`: An integer representing the number for which the 